In [20]:
# Copyright 2023, Qingwei Zhang. All Rights Reserved.
#
# MSCA 37014 3 Python for Analytics,
# Assignment 4: Churn analysis on Reddit
#
# Student: Qingwei Zhang
# Email: qingwei.zhang@uchicago.edu
# ==============================================================================

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import pyarrow.feather as feather


In [21]:
# Define data file location.
data_file_location = "./RC_2012_year_cohort.feather"

# read thefile, and the result is pandas.DataFrame
df = feather.read_feather(data_file_location)


In [22]:
# check the head of the df.
df.head()


,created_utc,author,subreddit
0,1325376000,irwinator,AskReddit
1,1325376000,reed311,politics
2,1325376000,sagapo3851,gaming
3,1325376000,filthgrinder,politics
4,1325376001,BitterDivorcedDad,WTF


In [23]:
# show the shape.
df.shape


(234225454, 3)

## 1. Find the top 30 subreddits

Count the number of comments in each subreddit, sort that list and pick the top 30.
While you are writing this code, it may be easier just to pick any three smaller sub-reddits. With smaller amount of data, you will be able to develop your code faster. Once you are sure the code works, switch to the top 30 sub-reddits.

In [24]:
# set how many subreddit from the top we want to see.
ranking = 30

df_ranking = df["subreddit"].value_counts(sort=True, ascending=False)[:ranking]

df_ranking


AskReddit              32023468
funny                  11909021
pics                    8973606
gaming                  6966306
WTF                     6377885
politics                5397547
atheism                 5043438
leagueoflegends         4029221
IAmA                    3883036
trees                   3846708
videos                  3611531
AdviceAnimals           3571534
todayilearned           2858174
fffffffuuuuuuuuuuuu     2785323
worldnews               2270210
gonewild                2168728
nfl                     2003374
movies                  1745437
starcraft               1729070
aww                     1691410
technology              1503424
Minecraft               1491322
soccer                  1339712
Music                   1146500
nba                     1116769
Diablo                  1100630
mylittlepony            1093572
Guildwars2              1080067
Games                   1068527
guns                    1024160
Name: subreddit, dtype: int64

In [25]:
# TODO, will remove, pick the last 3 subreddit.

reverse_df_ranking = df["subreddit"].value_counts(sort=True, ascending=True)[
    :ranking
]

reverse_df_ranking


trainerroad              1
VDI                      1
nod32                    1
RevivalGuild             1
onlinecommunities        1
paxskeptica              1
dacespicks               1
Fin                      1
serialkillersbookclub    1
EatSleepLift             1
WesternMovies            1
Toyota86                 1
CommonLawClass           1
whatdoesitallmean        1
clivecussler             1
BrownUniversity          1
yui                      1
BaconParadise            1
AppleSupport             1
Allergy                  1
ANIMALHELP               1
teflactivities           1
DWTP                     1
warpprism                1
TrueAnimeCSS             1
WTFRussia                1
dotnet30                 1
Rule_34                  1
ChildrensLit             1
HIV101course             1
Name: subreddit, dtype: int64

In [26]:
# Show the index name.
for i in range(ranking):
    print(i, df_ranking.index[i])


0 AskReddit
1 funny
2 pics
3 gaming
4 WTF
5 politics
6 atheism
7 leagueoflegends
8 IAmA
9 trees
10 videos
11 AdviceAnimals
12 todayilearned
13 fffffffuuuuuuuuuuuu
14 worldnews
15 gonewild
16 nfl
17 movies
18 starcraft
19 aww
20 technology
21 Minecraft
22 soccer
23 Music
24 nba
25 Diablo
26 mylittlepony
27 Guildwars2
28 Games
29 guns


## 2. Filter the dataframe so you are only looking at data form a single sub-reddit.

After this step, you should have a dataframe which only contains `author` and `created_utc` (you don't need the sub-reddit, because you are only working with a single sub-reddit at a time).

In [27]:
def extract_single_subreddit_info(ranking_df, current_index):
    single_subreddit = df.loc[
        df["subreddit"] == ranking_df.index[current_index]
    ]

    single_subreddit_df = single_subreddit[["created_utc", "author"]]

    return single_subreddit_df


In [28]:
index = 0

# TODO, code to be run at production.
# single_subreddit_info = extract_single_subreddit_info(df_ranking,index)

single_subreddit_info = extract_single_subreddit_info(
    df_ranking, index
)


## 3. Find the date of first interaction (for that sub-reddit)
Since we are looking to calculate metrics per sub-reddit, if `user123` 
posts a comment on 'gaming' and 'politics', we will consider them to be 
different authors.

Since our analysis will be done on a relative time basis (day 0 and day 1 
will be different for different users), calculate the date on which an author posted their first comment to that 
sub-reddit.

This will serve as our "day zero." You should end up with a dataframe 
containing the author and the date of the first comment.

In [29]:
# create a function to get all unique values of a columns
def get_unique_values(single_subreddit_info):
    #  get all Unique values of a columns as a list
    list_of_author = single_subreddit_info["author"].unique().tolist()

    # get the number of distinct values in a column.
    number_of_author = len(list_of_author)

    return list_of_author, number_of_author


In [40]:
# Create a function to find the date of first interaction.
def find_first_interaction(single_subreddit_info, iteration, list_of_author):
    # build a pandas DataFrame with a for-loop in Python
    rows = []

    for i in range(iteration):
        # get the day zero info for each author, it should be the
        # UTC  that has the minumum value. Since it is a single item
        # list object, we only need the first element.
        day_zero = (
            single_subreddit_info[
                single_subreddit_info["author"] == list_of_author[i]
            ]
            .min()
            .values[0]
        )

        # append to the list object.
        rows.append([list_of_author[i], day_zero])

    day_zero_df = pd.DataFrame(rows, columns=["author", "day_zero"])

    return day_zero_df


In [41]:
list_of_author, number_of_author = get_unique_values(single_subreddit_info)

print(f" {number_of_author} \n {list_of_author[:10]}")


 1125887 
 ['irwinator', 'AddictivePotential', 'Solumin', 'AliasSigma', '[deleted]', 'hecticengine', 'homer1969', 'rco8786', 'kagutsuchi', 'elliottblackwood']


In [42]:
test_df = find_first_interaction(single_subreddit_info, 20, list_of_author)

test_df


,author,day_zero
0,irwinator,1325376000
1,AddictivePotential,1325376005
2,Solumin,1325376006
3,AliasSigma,1325376012
4,[deleted],1325376013
5,hecticengine,1325376017
6,homer1969,1325376024
7,rco8786,1325376024
8,kagutsuchi,1325376025
9,elliottblackwood,1325376032


## 4. Combine the initial dataframe with the dataframe containing comment timestamps
After the second step, you had a dataframe containing the name of the author who commented and the time in which they commented. Combine this with the dataframe from step 2, where you created a dataframe which contains the name of the author who commented and the time when they _first__ commented (in that sub-reddit).

You can combine these two dataframes using the `merge` function (among other methods). At a high level, you are able to combine the two dataframes because they both contain the `author` column.

Note that we don't need hour/minute/second information. We are only concerned about their daily activity. You are welcome to use the `created_utc` field, as it exists, but this assignment will likely be easier if you use `pd.to_datetime` to convert to an actual `datetime` type.

You should now have a dataframe which contains the author's name, the date of their first comment and the date of their current comment.

Use the two dates in  your new dataframe to calculate the number of days since the first comment. 

You no longer need the original dates. You can do the rest of your analysis just with the author name and the column containing the days (since day zero) on which the author made a comment.

Note that many authors will comment in a subreddit multiple times in a day. We don't care about multiple comments. We are only care if they commented in a sub-reddit _at least_ once on that day.